# Pytorch Geometric GNN - homogen

## Import section

In [1]:
from data_gen import data_transform_split

In [2]:
import numpy as np
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
import pandas as pd

In [71]:
from torch_geometric.transforms import GCNNorm, NormalizeFeatures

## Section for GNN with rdkit

In [3]:
data_rdkit = data_transform_split(2)

D:\DATEN\Bachelorarbeit\data_gen.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['id_1']=df['aid'].map(lambda x: aid_translation_dictionary[x])
D:\DATEN\Bachelorarbeit\data_gen.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['id_2']=df['cid'].map(lambda x: cid_translation_dictionary[x])
D:\DATEN\Bachelorarbeit\data_gen.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

In [4]:
data_rdkit

Data(x=[457560, 208], train_pos_edge_index=[2, 979616], train_neg_edge_index=[2, 65612424], test_pos_edge_index=[2, 244898], test_neg_edge_index=[2, 16403118])

In [5]:
data_rdkit.num_features

208

In [7]:
data_rdkit.x

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [10.1531, -2.4522, 10.1531,  ...,  0.0000,  0.0000,  0.0000],
        [11.5979, -0.3191, 11.5979,  ...,  0.0000,  0.0000,  0.0000],
        [10.4979, -3.0391, 10.4979,  ...,  0.0000,  0.0000,  0.0000]])

In [9]:
data_rdkit.x.shape[0]

457560

In [65]:
(data.x.relu().sum(axis=0)==0).sum()

tensor(7)

In [28]:
new_column = torch.tensor(np.ones(data_rdkit.x.shape[0]), dtype=torch.float).reshape(1, -1).t()
new_column

tensor([[1.],
        [1.],
        [1.],
        ...,
        [1.],
        [1.],
        [1.]])

In [26]:
a = torch.rand(1,10)
b = a[0,:]
print(a)
print(b)
print(b.reshape(1, -1).t())

tensor([[0.2450, 0.7865, 0.2071, 0.1587, 0.3370, 0.3059, 0.6930, 0.9754, 0.0175,
         0.9674]])
tensor([0.2450, 0.7865, 0.2071, 0.1587, 0.3370, 0.3059, 0.6930, 0.9754, 0.0175,
        0.9674])
tensor([[0.2450],
        [0.7865],
        [0.2071],
        [0.1587],
        [0.3370],
        [0.3059],
        [0.6930],
        [0.9754],
        [0.0175],
        [0.9674]])


In [30]:
data_rdkit.x = torch.cat((data_rdkit.x, new_column), axis=1)

In [31]:
data_rdkit.x

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  1.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  1.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  1.0000],
        ...,
        [10.1531, -2.4522, 10.1531,  ...,  0.0000,  0.0000,  1.0000],
        [11.5979, -0.3191, 11.5979,  ...,  0.0000,  0.0000,  1.0000],
        [10.4979, -3.0391, 10.4979,  ...,  0.0000,  0.0000,  1.0000]])

In [12]:
data_rdkit.edge_index = torch.cat([data_rdkit.test_pos_edge_index, data_rdkit.train_pos_edge_index], dim=-1)

In [13]:
data_new = GCNNorm()(data_rdkit)
data_new

Data(x=[457560, 208], train_pos_edge_index=[2, 979616], train_neg_edge_index=[2, 65612424], test_pos_edge_index=[2, 244898], test_neg_edge_index=[2, 16403118], edge_index=[2, 1682074], edge_weight=[1682074])

In [16]:
(data_new.x!=data_rdkit.x).sum()

tensor(0)

In [72]:
data_new = NormalizeFeatures()(data_rdkit)
data_new

Data(x=[457560, 209], train_pos_edge_index=[2, 979616], train_neg_edge_index=[2, 65612424], test_pos_edge_index=[2, 244898], test_neg_edge_index=[2, 16403118])

In [99]:
torch.isnan(data_new.x).sum()

tensor(92)

In [101]:
data_rdkit.x = torch.tensor(np.random.rand(data_rdkit.x.shape[0], data_rdkit.x.shape[1]), dtype=torch.float)
data_rdkit.x

tensor([[0.8651, 0.1905, 0.9215,  ..., 0.9031, 0.3471, 0.4725],
        [0.7042, 0.3468, 0.1994,  ..., 0.2055, 0.0625, 0.3152],
        [0.0027, 0.2533, 0.9149,  ..., 0.6185, 0.4625, 0.7404],
        ...,
        [0.9810, 0.3969, 0.4471,  ..., 0.8952, 0.6479, 0.9719],
        [0.3596, 0.9181, 0.0828,  ..., 0.6332, 0.9475, 0.4834],
        [0.4147, 0.4871, 0.1751,  ..., 0.8589, 0.3981, 0.1067]])

## Setup pytorch Model

In [102]:
class GNN_homogen_with_data(torch.nn.Module):
    '''
    basic implementation of GNN, inspired by content shown in https://antoniolonga.github.io/Pytorch_geometric_tutorials/posts/post12.html from Antionio Longa
    '''
    def __init__(self):
        super(GNN_homogen_with_data, self).__init__()
        self.conv1 = GCNConv(data.num_features, 64) #int(data.num_features/2))
        self.conv2 = GCNConv(int(data.num_features/2), 64)
    
    def encode(self):
        x = self.conv1(data.x.relu(), data.train_pos_edge_index) # first convolution layer
        # x = x.relu() # relu function for tu - disables negative values
        # x = self.conv2(x, data.train_pos_edge_index) # second convolution layer
        return x
    
    def decode(self, z, pos_edge_index, neg_edge_index):
        edge_index = torch.cat([pos_edge_index, neg_edge_index], dim=-1) #fuse pos and neg edges together
        logits = (z[edge_index[0]] * z[edge_index[1]]).sum(dim=-1) # dot product of ... getting the mapped entries with z? z=model?
        return logits
    
    def decode_all(self, z):
        prob_adj = z @ z.t() # get adjacenticy matrix NxN
        return (prob_adj > 0).nonzero(as_tuple=False).t() # get predicted edge_list # should this not be 0.5?

In [103]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [104]:
data = data_rdkit.to(device)
model = GNN_homogen_with_data().to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)

In [105]:
def get_link_labels(pos_edge_index, neg_edge_index):
    # returns a tensor:
    # [1,1,1,1,...,0,0,0,0,0,..] with the number of ones is equel to the lenght of pos_edge_index
    # and the number of zeros is equal to the length of neg_edge_index
    E = pos_edge_index.size(1) + neg_edge_index.size(1)
    link_labels = torch.zeros(E, dtype=torch.float, device=device)
    link_labels[:pos_edge_index.size(1)] = 1.
    return link_labels

In [106]:
data.x

tensor([[0.8651, 0.1905, 0.9215,  ..., 0.9031, 0.3471, 0.4725],
        [0.7042, 0.3468, 0.1994,  ..., 0.2055, 0.0625, 0.3152],
        [0.0027, 0.2533, 0.9149,  ..., 0.6185, 0.4625, 0.7404],
        ...,
        [0.9810, 0.3969, 0.4471,  ..., 0.8952, 0.6479, 0.9719],
        [0.3596, 0.9181, 0.0828,  ..., 0.6332, 0.9475, 0.4834],
        [0.4147, 0.4871, 0.1751,  ..., 0.8589, 0.3981, 0.1067]])

In [135]:
model.train()

GNN_homogen_with_data(
  (conv1): GCNConv(209, 64)
  (conv2): GCNConv(104, 64)
)

In [136]:
optimizer.zero_grad()

In [137]:
z=model.encode()

In [138]:
z

tensor([[-9.6189e+00,  3.4387e+00, -5.4617e+00,  ..., -4.0006e+00,
         -3.6837e+00, -9.7379e+00],
        [-1.7464e+00,  7.2858e-01, -8.1092e-01,  ..., -7.0704e-01,
         -3.5331e-01, -1.6652e+00],
        [-1.1571e+00,  5.8220e-01, -3.9171e-01,  ..., -4.8766e-01,
         -4.4352e-01, -8.4365e-01],
        ...,
        [-4.9819e-01,  1.7642e-01, -3.0245e-01,  ..., -1.3701e-01,
          1.1615e-01, -1.1898e+00],
        [-6.1882e-01,  2.7655e-02,  6.5801e-02,  ..., -1.4631e-01,
         -2.6542e-03, -9.4102e-02],
        [-6.9829e-01,  2.7407e-01, -5.0496e-01,  ...,  2.5450e-01,
         -1.6318e-01, -5.0229e-01]], grad_fn=<AddBackward0>)

In [139]:
torch.isnan(z).sum()

tensor(0)

In [140]:
(z<0).sum()

tensor(16022971)

In [141]:
torch.isnan(z).sum()

tensor(0)

In [142]:
link_logits = model.decode(z, data.train_pos_edge_index, data.train_neg_edge_index)
link_logits

tensor([462.7568, 387.0594, 123.0425,  ..., 148.0059,  53.0981, 143.7039],
       grad_fn=<SumBackward1>)

In [143]:
torch.isnan(link_logits).sum()

tensor(0)

In [144]:
link_labels = get_link_labels(data.train_pos_edge_index, data.train_neg_edge_index)
link_labels

tensor([1., 1., 1.,  ..., 0., 0., 0.])

In [145]:
loss = F.binary_cross_entropy_with_logits(link_logits, link_labels)
loss

tensor(150.1187, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

In [146]:
torch.isnan(link_logits).sum()

tensor(0)

In [147]:
loss.backward()

In [148]:
optimizer.step()